# Techniques of High Performance Computing - Assignment 2

### Name: John Duffy

### Student Number: 19154676

In [132]:
# Import libraries common to Questions 1 & 2.

import numpy as np
import pyopencl as cl

# Question 1

## OpenCL CSR Matrix-Vector Product

**IMPORTANT NOTE**

My MacBook Pro is equipped with an Intel Core i5 CPU and an Intel Iris Plus Graphics GPU as depicted below.

    cl.get_platforms()[0].get_devices()

    [<pyopencl.Device 'Intel(R) Core(TM) i5-8279U CPU @ 2.40GHz' on 'Apple' at 0xffffffff>,
     <pyopencl.Device 'Intel(R) Iris(TM) Plus Graphics 655' on 'Apple' at 0x1024500>]
     
To ensure a context is created using the CPU for Question 1 (so that AVX2 technology is available) the function cl.Context() is used as below.

    ctx = cl.Context(dev_type = cl.device_type.CPU)  # Tell OpenCL to use the CPU device.
    
The function cl.get_some_context() used in the course lecture notes is not specific enough (on my MacBook Pro at least) which then causes kernel build problems.

### Program Description

This program...


### Optimisation Results

A class LinearOperatorBaseline was created to use as a baseline for measuring the performance gains through subsequnt use of OpenCL and AVX2.

In [133]:
from scipy.sparse import csr_matrix, eye
from scipy.sparse.linalg import LinearOperator

In [134]:
# Define the class LinearOperatorBaseline for baseline performance measurements.

class LinearOperatorBaseline(LinearOperator):
    """
    This class... 
    """
    
    def __init__(self, data, indices, indptr):
        """
        """
        self.data = data
        self.indices = indices
        self.indptr = indptr
        self.shape = (len(indptr) - 1, len(indptr) - 1)  # Assume square.
    
    def _matvec(self, x):
        """
        """
        y = np.zeros(x.shape[0], dtype=np.float32)
        
        for i in range(self.shape[1]): 
            y[i] = np.dot(self.data[self.indptr[i]:self.indptr[i + 1]], x[self.indices[self.indptr[i]]:self.indptr[i + 1]])
            
        return y

In [135]:
# Create a test instance of LinearOperatorBaseline.

N = 1000000

csr = eye((N), dtype=np.float32).tocsr()

linear_operator = LinearOperatorBaseline(csr.data, csr.indices, csr.indptr)

v = np.full((N), 2)

In [136]:
%%timeit

linear_operator.matvec(v)

2.74 s ± 115 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [137]:
%%timeit

linear_operator * v

2.55 s ± 10.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [138]:
linear_operator.matvec(v)

array([2., 2., 2., ..., 2., 2., 2.], dtype=float32)

In [139]:
linear_operator * v

array([2., 2., 2., ..., 2., 2., 2.], dtype=float32)

In [140]:
# Define the class LinearOperatorOpenCL.

class LinearOperatorOpenCL(LinearOperator):
    """
    This class... 
    """
    
    def __init__(self, data, indices, indptr):
        """
        """
        self.shape = (len(indptr) - 1, len(indptr) - 1)  # Assume square.

        #self.data = data
        #self.indices = indices
        #self.indptr = indptr
        
        #self.ctx = cl.create_some_context()
        #self.queue = cl.CommandQueue(self.ctx)
        
        #mf = cl.mem_flags
 
        # Create device buffers for the sparse matrix data and copy to the device.

        #self.device_global_data = cl.Buffer(self.ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = data)
        #self.device_global_indices = cl.Buffer(self.ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = indices)       
        #self.device_global_indptr = cl.Buffer(self.ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = indptr)

        # Create device buffers for v and y. Data will be copied to/from these buffer later.
        
        #self.program = cl.Program(self.ctx, """
        #__kernel void matvec_opencl(__global const float *v, __global float *y)
        #{
        #int gid = get_global_id(0);
        #    y[gid] = v[gid];
        #}
        #""").build()

    
    def _matvec(self, v):
        """
        """

        mf = cl.mem_flags

        
        #ctx = cl.create_some_context() # Not specific enough!

        #platform = cl.get_platforms()[0]
        #device   = platform.get_devices()[1] # Get the GPU ID
        #ctx      = cl.Context([device])      # Tell CL to use GPU

        # TODO: WRONG DEVICE TYPE, NEED CPU FOR AVX2.
        
        ctx = cl.Context(dev_type = cl.device_type.GPU)      # Tell CL to use CPU
        
        queue = cl.CommandQueue(ctx)
        
        program = cl.Program(ctx, """
        __kernel void matvec_opencl(__global const float *v, __global float *y)
        {
            int gid = get_global_id(0);
            y[gid] = 3 * v[gid];
        }
        """).build()


        host_v = np.ones((1000), dtype=np.float32)

        host_y = np.zeros((1000), dtype=np.float32)

        device_global_v = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = host_v)

        device_global_y = cl.Buffer(ctx, mf.WRITE_ONLY | mf.COPY_HOST_PTR, hostbuf = host_y)
                
        program.matvec_opencl(queue, (1000,), None, device_global_v, device_global_y)
        
        cl.enqueue_copy(queue, host_y, device_global_y)
        
               
        #for i in range(self.shape[1]): 
        #y[i] = np.dot(self.data[self.indptr[i]:self.indptr[i + 1]], x[self.indices[self.indptr[i]]:self.indptr[i + 1]])
            
        return host_y


In [141]:
# Create a test instance of LinearOperatorOpenCL.

N = 1000

csr = eye((N), dtype=np.float32).tocsr()

linear_operator = LinearOperatorOpenCL(csr.data, csr.indices, csr.indptr)

v = np.full((N), 2)

In [142]:
# TEST

linear_operator.matvec(v)



array([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3.

In [143]:
%%timeit

linear_operator.matvec(v)

8.08 ms ± 570 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [144]:
%%timeit

linear_operator * v

7.97 ms ± 368 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [145]:
linear_operator.matvec(v)

array([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3.

In [146]:
linear_operator * v

array([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3.

# Question 2

## Solving a Poisson Problem with OpenCL

**IMPORTANT NOTE**

My MacBook Pro is equipped with an Intel Core i5 CPU and an Intel Iris Plus Graphics GPU as depicted below.

    cl.get_platforms()[0].get_devices()

    [<pyopencl.Device 'Intel(R) Core(TM) i5-8279U CPU @ 2.40GHz' on 'Apple' at 0xffffffff>,
     <pyopencl.Device 'Intel(R) Iris(TM) Plus Graphics 655' on 'Apple' at 0x1024500>]
     
To ensure a context is created using the GPU for Question 2 the function cl.Context() is used as below.

    ctx = cl.Context(dev_type = cl.device_type.GPU)  # Tell OpenCL to use the GPU device.
    
The function cl.get_some_context() used in the course lecture notes is not specific enough (on my MacBook Pro at least) which then causes kernel build problems.

### Program Description

This program...

In [147]:
from scipy.sparse.linalg import cg 

In [148]:
cl.get_platforms()[0].get_devices()

[<pyopencl.Device 'Intel(R) Core(TM) i5-8279U CPU @ 2.40GHz' on 'Apple' at 0xffffffff>,
 <pyopencl.Device 'Intel(R) Iris(TM) Plus Graphics 655' on 'Apple' at 0x1024500>]

In [149]:
ctx = cl.Context(dev_type = cl.device_type.CPU)

In [150]:
ctx.get_info(cl.context_info.DEVICES)

[<pyopencl.Device 'Intel(R) Core(TM) i5-8279U CPU @ 2.40GHz' on 'Apple' at 0xffffffff>]